In [22]:
import pandas as pd
import re
import numpy as np


In [23]:
ruta = 'comentarios_Alvaro Delgado Uruguay_procesado_Ortografiado_sentimiento_analizado'
df = pd.read_csv(f'{ruta}.csv')

In [24]:
df.sample(2
        )

,Comentario,Autor,Likes_Comentario,Fecha_Publicacion_Comentario,URL_Video,Titulo_Video,Descripcion_Video,Tags,Fecha_Publicacion_Video,Idioma,...,Comentario_nrc,Titulo_Video_afinn,Titulo_Video_nrc,Descripcion_Video_afinn,Descripcion_Video_nrc,Comentario_Polaridad,Comentario_Subjetividad,Tags_afinn,Tags_nrc,Categoria_Nombre
3746,Plata desapareció,@piglinslayer5159,7,2024-07-22T22:08:26Z,https://www.youtube.com/watch?v=NMf3uKHuTTs,álvaro delgado pidió renuncia intendente artig...,tras ser condenado caso investiga cobro irregu...,NaN,2024-07-22T17:27:10Z,es-419,...,NaN,-1,"positivo:1, negativo:1",-4,"miedo:2, negativo:4, tristeza:2, positivo:4, a...",0.00,0.000,0,NaN,News & Politics
2638,"Buen reporte gracias, votante orgullosa ser ur...",@soguma2011,11,2024-10-28T02:27:04Z,https://www.youtube.com/watch?v=lxkUpiOCM9k,ballottage uruguay segunda vuelta candidatos d...,millones ciudadanos uruguayos asisten domingo ...,canal noticias canal argentina canal noticias ...,2024-10-28T01:38:41Z,es-419,...,"anticipación:2, alegría:3, positivo:3, sorpres...",0,positivo:1,-1,"enfado:1, anticipación:6, alegría:2, negativo:...",0.52,0.645,0,asco:4,News & Politics


# Rescatar Tags, Paises y Emociones

In [25]:
from collections import Counter
# Inicializa un contador para sumar las emociones
contador_emociones = Counter()

# Recorremos cada fila de la columna
for fila in df['Comentario_nrc'].dropna():
    emociones = fila.split(',')  # Separar por coma
    for emocion in emociones:
        emocion = emocion.strip()
        if ':' in emocion:
            clave, valor = emocion.split(':')
            clave = clave.strip().lower()
            try:
                valor = int(valor.strip())
                contador_emociones[clave] += valor
            except ValueError:
                continue  # Ignora si no puede convertir el valor

# Convertimos el contador a DataFrame ordenado
resumen_df = pd.DataFrame(contador_emociones.items(), columns=['Emocion', 'Total'])
resumen_df = resumen_df.sort_values(by='Total', ascending=False)

# Guardar el resumen como CSV
resumen_df.to_csv('resumen_emociones_nrc.csv', index=False, encoding='utf-8-sig')

print("✅ Resumen guardado en 'resumen_emociones_nrc.csv'")

✅ Resumen guardado en 'resumen_emociones_nrc.csv'


In [26]:
from collections import Counter
# 2. Procesar columna Tags y contar los más frecuentes
def contar_tags(df, columna='Tags'):
    todos_los_tags = []

    for tags in df[columna].dropna():
        # Separar tags por espacio
        separados = tags.split()
        todos_los_tags.extend(separados)

    # Contar frecuencia con Counter
    conteo = Counter(todos_los_tags)
    df_resultado = pd.DataFrame(conteo.items(), columns=['Tag', 'Frecuencia'])
    df_resultado = df_resultado.sort_values(by='Frecuencia', ascending=False).reset_index(drop=True)
    return df_resultado


In [27]:
print(df['Pais_Canal'].unique())
conteo_paises = df['Pais_Canal'].value_counts().reset_index()
conteo_paises.columns = ['País', 'Frecuencia']  # Renombrar columnas
conteo_paises['Porcentaje'] = (conteo_paises['Frecuencia'] / len(df)) * 100  # Añadir porcentaje

print(conteo_paises)

['UY' 'AR' nan 'US' 'PE' 'FR' 'ES' 'DE' 'IN' 'CO' 'EC' 'MX' 'VE']
   País  Frecuencia  Porcentaje
0    UY        3642   68.484393
1    AR        1014   19.067319
2    US         215    4.042873
3    FR         136    2.557352
4    DE          86    1.617149
5    ES          29    0.545318
6    CO          19    0.357277
7    IN           3    0.056412
8    MX           3    0.056412
9    EC           2    0.037608
10   PE           1    0.018804
11   VE           1    0.018804


In [28]:
df_tags = contar_tags(df)
print("\n🎯 Tags más frecuentes:\n")
print(df_tags.head(20))  
# Guardar como CSV en la misma carpeta del notebook
df_tags.to_csv("tags_contados.csv", index=False)
print("✅ Archivo CSV guardado como: tags_contados.csv")



🎯 Tags más frecuentes:

           Tag  Frecuencia
0      uruguay       10079
1       debate        8669
2      delgado        4286
3         orsi        4076
4     analisis        3364
5       alvaro        3240
6         vivo        3216
7        pisso        2103
8       franco        2103
9    encuestas        1882
10  elecciones        1878
11     yamandu        1512
12    noticias        1327
13  presidente        1200
14     yamandú        1082
15    balotaje         948
16  observador         715
17    análisis         709
18          uy         701
19        pais         701
✅ Archivo CSV guardado como: tags_contados.csv


## Viztazo a los Datos: reporte_datos.txt


In [29]:
def analizar_comentarios(df, log):
    log.append("\n🔍 Tipo real de objetos en la columna 'Comentario':")
    tipos = df['Comentario'].map(type).value_counts()
    log.append(str(tipos))

    log.append("\n⚠️ Ejemplos de valores no str (si existen):")
    no_str = df[~df['Comentario'].map(lambda x: isinstance(x, str))]
    log.append(str(no_str.head()))

    

def analizar_calidad_datos(df, log):
    log.append("\n📊 INFORME GENERAL DEL DATAFRAME")
    log.append(f"Filas: {df.shape[0]}, Columnas: {df.shape[1]}")

    log.append("\n🧠 Tipos de datos por columna:")
    log.append(str(df.dtypes))

    log.append("\n🔎 Valores únicos por columna:")
    log.append(str(df.nunique()))

    log.append("\n🚫 Conteo de valores nulos (NaN) por columna:")
    log.append(str(df.isna().sum()))

    log.append("\n📉 Porcentaje de valores nulos por columna:")
    log.append(str((df.isna().mean() * 100).round(2).astype(str) + '%'))


def guardar_log(log):
    with open("reporte_datos.txt", "w", encoding="utf-8") as f:
        for linea in log:
            f.write(str(linea) + "\n")
    print("📄 Informe guardado como 'reporte_datos.txt'")

In [30]:
log = []
analizar_calidad_datos(df, log)
if 'Comentario' in df.columns:
    analizar_comentarios(df, log)
else:
    log.append("\n❌ La columna 'Comentario' no está en el archivo.")

guardar_log(log)

📄 Informe guardado como 'reporte_datos.txt'


## PreProcesamiento de Datos para Machine Learning: ML

| **Columna**                    | **¿Usable?** | **Cómo procesarla / convertirla**                                 |
| ------------------------------ | ------------ | ----------------------------------------------------------------- |
| `Comentario`                   | ✅            | NLP: limpiar, tokenizar, vectorizar (TF-IDF, embeddings)          |
| `Autor`                        | 🔸           | Poco útil directamente; posible usar como frecuencia si se repite |
| `Likes_Comentario`             | ✅            | Escalar (MinMaxScaler o StandardScaler)                           |
| `Fecha_Publicacion_Comentario` | ✅            | Extraer año, mes, día, hora como nuevas columnas                  |
| `URL_Video`                    | ❌            | No útil para ML; solo identificador                               |
| `Titulo_Video`                 | ✅            | NLP: igual que `Comentario`                                       |
| `Descripcion_Video`            | ✅            | NLP: igual que `Comentario`                                       |
| `Tags`                         | ✅            | Tokenizar, contar palabras clave, binarizar o vectorizar          |
| `Fecha_Publicacion_Video`      | ✅            | Extraer año, mes, hora para análisis temporal                     |
| `Idioma`                       | ✅            | Codificar como categórica (One-Hot o LabelEncoding)               |
| `Duracion`                     | ✅            | Convertir de ISO 8601 (`PT2H50M25S`) a minutos totales            |
| `Numero_Views`                 | ✅            | Escalar                                                           |
| `Numero_Likes`                 | ✅            | Escalar                                                           |
| `Numero_Comentarios`           | ✅            | Escalar                                                           |
| `Pais_Canal`                   | ✅            | Codificar país (categórica)                                       |
| `Subscriptores_Canal`          | ✅            | Escalar                                                           |
| `Canal_Titulo`                 | 🔸           | Útil si se agrupa o analiza como canal dominante                  |
| `Comentario_afinn`             | ✅            | Escalar (sentimiento ya numerizado)                               |
| `Comentario_nrc`               | ✅            | Contar emociones y expandir como columnas: alegría, miedo, etc.   |
| `Titulo_Video_afinn`           | ✅            | Escalar                                                           |
| `Titulo_Video_nrc`             | ✅            | Igual que `Comentario_nrc`                                        |
| `Descripcion_Video_afinn`      | ✅            | Escalar                                                           |
| `Descripcion_Video_nrc`        | ✅            | Igual que `Comentario_nrc`                                        |
| `Comentario_Polaridad`         | ✅            | Escalar (valor entre -1 y 1)                                      |
| `Comentario_Subjetividad`      | ✅            | Escalar (valor entre 0 y 1)                                       |
| `Tags_afinn`                   | ✅            | Escalar                                                           |
| `Tags_nrc`                     | ✅            | Expandir emociones como columnas                                  |
| `Categoria_Nombre`             | ✅            | Codificar como variable categórica                                |


In [31]:

# --- Eliminar columnas específicas ---
def eliminar_columnas(df, columnas_a_eliminar, log):
    columnas_eliminadas = [col for col in columnas_a_eliminar if col in df.columns]
    df = df.drop(columns=columnas_eliminadas, errors='ignore')
    log.append(f"🗑️ Columnas eliminadas: {columnas_eliminadas}")
    return df
def ponderar_pais(df, columna):
    total_paises = df[columna].dropna().value_counts(normalize=True)
    ponderacion = df[columna].map(total_paises).fillna(0)
    return ponderacion
def parsear_duracion_iso8601(duracion):
    if not isinstance(duracion, str) or not duracion.startswith('PT'):
        return 0
    horas = minutos = segundos = 0

    match_h = re.search(r'(\d+)H', duracion)
    match_m = re.search(r'(\d+)M', duracion)
    match_s = re.search(r'(\d+)S', duracion)

    if match_h:
        horas = int(match_h.group(1))
    if match_m:
        minutos = int(match_m.group(1))
    if match_s:
        segundos = int(match_s.group(1))

    total_segundos = horas * 3600 + minutos * 60 + segundos
    return total_segundos
def ponderar_categoria(df):
    # Diccionario de pesos por categoría PONDERACION OBTENIDA GRACIAS A CHATGPT 4o
    pesos_categoria = {
        "News & Politics": 1.0,
        "Education": 0.8,
        "People & Blogs": 0.7,
        "Entertainment": 0.6,
        "Comedy": 0.6,
        "Music": 0.5,
        "Howto & Style": 0.5,
        "Science & Technology": 0.7,
        "Sports": 0.4,
        "Gaming": 0.3,
        "Film & Animation": 0.4,
        "Autos & Vehicles": 0.2,
        "Pets & Animals": 0.2,
        "Travel & Events": 0.3,
        "Videoblogging": 0.5,
        "Short Movies": 0.4,
        "Movies": 0.3,
        "Nonprofits & Activism": 0.6,
        "Anime/Animation": 0.3,
        "Action/Adventure": 0.3,
        "Classics": 0.3,
        "Comedy (Movies)": 0.5,
        "Documentary": 0.7,
        "Drama": 0.4,
        "Family": 0.4,
        "Foreign": 0.3,
        "Horror": 0.3,
        "Sci-Fi/Fantasy": 0.3,
        "Thriller": 0.3,
        "Shorts": 0.4,
        "Shows": 0.4,
        "Trailers": 0.2
    }

    # Asigna el peso según la categoría (NaN si no está en el diccionario)
    df['Categoria_Ponderada'] = df['Categoria_Nombre'].map(pesos_categoria)

    # Rellena valores faltantes con un peso neutro o bajo, por ejemplo 0.3
    df['Categoria_Ponderada'] = df['Categoria_Ponderada'].fillna(0.3)

    # Elimina la columna original
    df.drop(columns=['Categoria_Nombre'], inplace=True)

    return df

# --- Rellenar valores nulos ---
def rellenar_nulos(df, log):
    log.append("\n🧽 Relleno de valores nulos:")
    for col in df.columns:
        if df[col].isna().sum() > 0:
            if df[col].dtype == 'object':
                if df[col].nunique() < 30:
                    modo = df[col].mode(dropna=True)
                    valor = modo[0] if not modo.empty else "desconocido"
                    df[col] = df[col].fillna(valor)
                    log.append(f"  - {col}: categórica → rellenado con '{valor}'")
                else:
                    df[col] = df[col].fillna("")
                    log.append(f"  - {col}: texto → rellenado con cadena vacía")
            elif np.issubdtype(df[col].dtype, np.number):
                mediana = df[col].median()
                df[col] = df[col].fillna(mediana)
                log.append(f"  - {col}: numérica → rellenado con mediana ({mediana})")
    return df

# --- Expandir columnas _nrc a columnas de emociones individuales ---
def expandir_emociones_nrc(df, log):
    columnas_nrc = [col for col in df.columns if col.endswith('_nrc')]
    emociones_total = set()

    log.append("\n🌈 Expansión de columnas _nrc:")
    for col in columnas_nrc:
        log.append(f"  - Procesando columna: {col}")
        emociones_dicts = df[col].fillna("").apply(lambda x: {
            k.strip(): int(v) for k, v in (item.split(":") for item in x.split(",") if ":" in item)
        })

        emociones_col = set()
        for dic in emociones_dicts:
            emociones_col.update(dic.keys())
        emociones_total.update(emociones_col)

        for emocion in emociones_col:
            df[f"{col}_{emocion}"] = emociones_dicts.apply(lambda d: d.get(emocion, 0))

        df.drop(columns=[col], inplace=True)

    log.append(f"  - Emociones detectadas: {sorted(list(emociones_total))}")
    return df

# --- Guardar resumen del procesamiento ---
def guardar_resumen(log, ruta_salida_base):
    resumen_path = f"{ruta_salida_base}_resumen_procesamiento.txt"
    with open(resumen_path, "w", encoding="utf-8") as f:
        for linea in log:
            f.write(linea + "\n")
    print(f"📄 Resumen guardado como: {resumen_path}")

### Manejo de fechas

In [32]:

# Asegúrate de que la columna sea tipo datetime
df['Fecha_Publicacion_Comentario'] = pd.to_datetime(df['Fecha_Publicacion_Comentario'], errors='coerce')

# 1. Extraer componentes temporales (útiles para estacionalidad y análisis temporal)
df['comentario_anio'] = df['Fecha_Publicacion_Comentario'].dt.year
df['comentario_mes'] = df['Fecha_Publicacion_Comentario'].dt.month
df['comentario_dia'] = df['Fecha_Publicacion_Comentario'].dt.day
df['comentario_dia_semana'] = df['Fecha_Publicacion_Comentario'].dt.weekday  # Lunes = 0
df['comentario_hora'] = df['Fecha_Publicacion_Comentario'].dt.hour

# 2. Convertir a timestamp numérico (útil para modelos que capturan tendencia temporal)
# Esto convierte la fecha a segundos desde 1970-01-01 (Unix Epoch)
df['comentario_timestamp'] = df['Fecha_Publicacion_Comentario'].astype('int64') // 10**9

# (Opcional) Revisar si alguna fila no pudo convertirse correctamente
errores_fecha = df['Fecha_Publicacion_Comentario'].isnull().sum()
if errores_fecha > 0:
    print(f"⚠️ Atención: {errores_fecha} valores no pudieron convertirse a datetime.")


In [33]:
##PARA REDES NEURONALES O REGRESION LINEAL
'''from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df['comentario_timestamp_norm'] = scaler.fit_transform(df[['comentario_timestamp']])'''


"from sklearn.preprocessing import MinMaxScaler\nscaler = MinMaxScaler()\ndf['comentario_timestamp_norm'] = scaler.fit_transform(df[['comentario_timestamp']])"

In [34]:
# --- BLOQUE DE EJECUCIÓN PARA NOTEBOOK ---

log = [f"📂 Procesando DataFrame desde ruta: {ruta}.csv"]

# Paso 1: Eliminar columnas no deseadas
columnas_a_eliminar = [
    'Comentario', 'Titulo_Video', 'Descripcion_Video',
    'Fecha_Publicacion_Video', 'Idioma','Autor','URL_Video','Tags','Canal_Titulo','Fecha_Publicacion_Comentario'
]
df = eliminar_columnas(df, columnas_a_eliminar, log)

# Paso 2: Rellenar valores nulos
df = rellenar_nulos(df, log)
# Ponderar países
df['Ponderacion_Pais_Canal'] = ponderar_pais(df, 'Pais_Canal')
df = df.drop(columns=['Pais_Canal'], errors='ignore')
# Paso 3: Expandir emociones NRC
df = expandir_emociones_nrc(df, log)
df= ponderar_categoria(df)
# Parsear Duracion
df['Duracion_Segundos'] = df['Duracion'].apply(parsear_duracion_iso8601)
df = df.drop(columns=['Duracion'])  # opcional: elimina columna original
# Paso 4: Guardar CSV y resumen
ruta_salida_base = ruta + "_ML"
df.to_csv(f"{ruta_salida_base}.csv", index=False, encoding='utf-8-sig')
log.append(f"\n💾 Archivo final guardado como: {ruta_salida_base}.csv")

guardar_resumen(log, ruta_salida_base)

print("✅ Procesamiento finalizado.")


📄 Resumen guardado como: comentarios_Alvaro Delgado Uruguay_procesado_Ortografiado_sentimiento_analizado_ML_resumen_procesamiento.txt
✅ Procesamiento finalizado.


# Features Derivados recomendados

Ponderación del Comentario

Ponderador = (Likes_Comentario + 1) / (Numero_Comentarios + 1)
→ Más peso a comentarios más relevantes dentro de su contexto.

Engagement del Video

Engagement = (Numero_Likes + Numero_Comentarios) / (Numero_Views + 1)
→ Refleja cuánto interactúa el público con ese video.

Influencia del Canal

Influencia_Canal = log(Subscriptores_Canal + 1)
→ Escala logarítmica para evitar distorsión por canales gigantes.

Polaridad ponderada

Comentario_Polaridad_Ponderada = Comentario_Polaridad * Ponderador

Subjetividad ponderada

Comentario_Subjetividad_Ponderada = Comentario_Subjetividad * Ponderador

AFINN ponderado

Comentario_afinn_Ponderado = Comentario_afinn * Ponderador

Emoción codificada (One-hot o embedding)
→ Convertir Comentario_nrc_emocion a variables binarias como:

emocion_positivo, emocion_miedo, emocion_confianza, etc.

In [35]:


def crear_features_avanzados(df):
    # Prevenir división por cero
    df['Likes_Comentario'] = df['Likes_Comentario'].fillna(0)
    df['Numero_Comentarios'] = df['Numero_Comentarios'].replace(0, np.nan)
    df['Numero_Views'] = df['Numero_Views'].replace(0, np.nan)
    df['Numero_Likes'] = df['Numero_Likes'].replace(0, np.nan)
    df['Subscriptores_Canal'] = df['Subscriptores_Canal'].replace(0, np.nan)

    # --- 1. Ponderador del Comentario ---
    df['Ponderador_Comentario'] = (df['Likes_Comentario'] + 1) / (df['Numero_Comentarios'] + 1)

    # --- 2. Engagement del Video ---
    df['Engagement_Video'] = (df['Numero_Likes'] + df['Numero_Comentarios']) / (df['Numero_Views'] + 1)

    # --- 3. Influencia del Canal ---
    df['Influencia_Canal'] = np.log1p(df['Subscriptores_Canal'])  # log(Subscriptores_Canal + 1)

    # --- 4. Polaridad y Subjetividad Ponderadas ---
    df['Comentario_Polaridad_Ponderada'] = df['Comentario_Polaridad'] * df['Ponderador_Comentario']
    df['Comentario_Subjetividad_Ponderada'] = df['Comentario_Subjetividad'] * df['Ponderador_Comentario']

    # --- 5. AFINN Ponderado ---
    df['Comentario_afinn_Ponderado'] = df['Comentario_afinn'] * df['Ponderador_Comentario']
    df['Titulo_Video_afinn_Ponderado'] = df['Titulo_Video_afinn'] * df['Engagement_Video']
    df['Descripcion_Video_afinn_Ponderado'] = df['Descripcion_Video_afinn'] * df['Engagement_Video']
    df['Tags_afinn_Ponderado'] = df['Tags_afinn'] * df['Influencia_Canal']

    # --- 6. Emociones NRC Ponderadas ---
    emociones = [
        'alegría', 'anticipación', 'enfado', 'tristeza', 'positivo', 'sorpresa',
        'asco', 'negativo', 'confianza', 'miedo'
    ]
    fuentes = ['Comentario_', 'Titulo_Video_', 'Descripcion_Video_', 'Tags_']

    for emocion in emociones:
        for fuente in fuentes:
            col = f'{fuente}nrc_{emocion}'
            if col in df.columns:
                nueva_col = f'{col}_Ponderada'
                if fuente == 'Comentario_':
                    df[nueva_col] = df[col] * df['Ponderador_Comentario']
                elif fuente == 'Titulo_Video_' or fuente == 'Descripcion_Video_':
                    df[nueva_col] = df[col] * df['Engagement_Video']
                else:
                    df[nueva_col] = df[col] * df['Influencia_Canal']

    # --- 7. Rellenar nulos generados ---
    df.fillna(0, inplace=True)

    return df


In [36]:
df_features= crear_features_avanzados(df)
df_features.sample(10)

C:\Users\HP\AppData\Local\Temp\ipykernel_13876\2504438377.py:43: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[nueva_col] = df[col] * df['Engagement_Video']
C:\Users\HP\AppData\Local\Temp\ipykernel_13876\2504438377.py:45: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[nueva_col] = df[col] * df['Influencia_Canal']
C:\Users\HP\AppData\Local\Temp\ipykernel_13876\2504438377.py:41: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Co

,Likes_Comentario,Numero_Views,Numero_Likes,Numero_Comentarios,Subscriptores_Canal,Comentario_afinn,Titulo_Video_afinn,Descripcion_Video_afinn,Comentario_Polaridad,Comentario_Subjetividad,...,Descripcion_Video_nrc_negativo_Ponderada,Tags_nrc_negativo_Ponderada,Comentario_nrc_confianza_Ponderada,Titulo_Video_nrc_confianza_Ponderada,Descripcion_Video_nrc_confianza_Ponderada,Tags_nrc_confianza_Ponderada,Comentario_nrc_miedo_Ponderada,Titulo_Video_nrc_miedo_Ponderada,Descripcion_Video_nrc_miedo_Ponderada,Tags_nrc_miedo_Ponderada
3624,3,3735,77.0,18,65500,1,0,0,0.000000,0.000000,...,0.000000,0.0,0.000000,0.025428,0.000000,0.000000,0.000000,0.00000,0.000000,0.0
4303,4,62166,2575.0,778,140000,0,0,0,-0.166667,0.066667,...,0.000000,0.0,0.019255,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0
3857,0,824,12.0,14,118000,0,0,12,-0.600000,0.800000,...,0.094545,0.0,0.000000,0.000000,0.378182,0.000000,0.000000,0.00000,0.031515,0.0
4107,0,62166,2575.0,778,140000,-3,0,0,-0.500000,1.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0
204,0,142420,13421.0,1294,1190000,2,-1,0,0.100000,0.350000,...,0.000000,0.0,0.000000,0.000000,0.000000,69.947324,0.000000,0.10332,0.000000,0.0
1406,3,6826,73.0,48,118000,0,0,15,0.000000,0.100000,...,0.354475,0.0,0.081633,0.000000,0.425370,0.000000,0.000000,0.00000,0.177237,0.0
926,0,54039,611.0,527,56400,-1,0,8,0.600000,0.900000,...,0.252702,0.0,0.000000,0.000000,0.231643,0.000000,0.001894,0.00000,0.126351,0.0
1459,1,6714,129.0,87,56400,0,0,0,0.000000,0.000000,...,0.096500,0.0,0.000000,0.000000,0.257334,0.000000,0.000000,0.00000,0.096500,0.0
253,1,142420,13421.0,1294,1190000,0,-1,0,0.000000,0.500000,...,0.000000,0.0,0.003089,0.000000,0.000000,69.947324,0.004633,0.10332,0.000000,0.0
5240,0,26467,312.0,442,3280,9,0,8,0.203030,0.324242,...,0.085462,0.0,0.013544,0.028487,0.256385,32.383614,0.011287,0.00000,0.056974,0.0


In [37]:
ruta_salida_fealtures = ruta_salida_base + "_fealtures"
df_features.to_csv(f"{ruta_salida_fealtures}.csv", index=False, encoding='utf-8-sig')
print(f"\n💾 Archivo final guardado como: {ruta_salida_fealtures}.csv")


💾 Archivo final guardado como: comentarios_Alvaro Delgado Uruguay_procesado_Ortografiado_sentimiento_analizado_ML_fealtures.csv


In [38]:

contador = 0
for columna in df.columns:
    if contador >= 5:  # Detener después de 5 columnas
        break
        
    print(f"Columna: {columna}")
    print(df[columna].unique())  # Valores únicos
    print("-" * 50)  # Separador visual
    contador += 1  # Incrementar el contador

print("\nSe han mostrado las primeras 5 columnas del DataFrame.")

Columna: Likes_Comentario
[  0   1   5   4  10   2   3   8  11  12  15  13   6   9  14  21  23  27
   7 188  29  28  87  44 100  45  18  17  22  19  16 355  75  51  41  39
  32 130  42  49  31  34  20  57  40 103  43  47  38  24  25  30  26  33
  35  46  37  54  36  99  48  66  65 158  71  58  98  81  94 231 102 107
  83  64  55  84  50 113  56  60  76]
--------------------------------------------------
Columna: Numero_Views
[440876 495113 386637 142420 177523  81856  84217  54039  42812  26467
  25509  31447  44985  11821  10707  12298   6826   6714   5674   4513
   4343   4200   3682   3660   3488   3455   3371   3123   2721   2702
   2681   2940   2576   1921   1724   1488   1263   1258   1311   1248
   1174   1083   1062   1011    862    839 702736 125764  96543  75373
  30455  20242  18052  16329  16008  14335  12359  11416  10969  10157
  10140   9404   8761   8636   8177   8033   7583   7339   7302   6453
   6390   6233   6190   5456   4893   4636   4623  41809   4293   4203
   